<a href="https://colab.research.google.com/github/KwakuBonfulBosompim/MSc-Data-Analytics-and-ML-Projects/blob/main/Teaching_a_Neural_Network_to_Spot_Fake_Bills.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The neural network model or algorithm is like a team of tiny brain cells. Each cell learns a little rule, and together they decide if a bill is real or fake. We feed the network lots of example bills, it practices, and then it gets good at guessing new bills

We will use bill_authentication.csv dataset where the last column is the label (0 = fake, 1 = real) and the other columns are numeric features.

First We Import relevant libraries(group of modules or functions) & load data

In [1]:
import pandas as pd # reads and handles data tables (like Excel in Python)
import numpy as np # does maths with numbers and arrays
import tensorflow as tf # helps us build neural networks
from tensorflow import keras # keras is a way to build neural networks inside TensorFlow
from tensorflow.keras import layers # the layers is like building blocks of a neural network, like Lego pieces 🧱
from sklearn.model_selection import train_test_split # split our data into training set (to teach) and testing set (to check learning)
from sklearn.preprocessing import StandardScaler # used to scale/normalize data so everything is on the same range. (it has a mathematical formula)
from sklearn.metrics import confusion_matrix, classification_report # check how well our model did (like a scorecard 📊)

In [2]:
# Reproducibility
# it's like always use the same sequence of random numbers.”
# This makes the experiment repeatable — if you run it again, you’ll get the same train/test split, same neural network initialization, and same result
np.random.seed(42)
tf.random.set_seed(42)
# so here we our experiment repeatable. If we run it again, we get the same random results (like locking the dice 🎲 so it always rolls the same way).

# Machine learning uses randomness, like Picking random rows for train/test split, Initializing random weights in the neural network, Shuffling data during training
# so we control this randomness, because if we don't every time you run your code you might get slightly different results.
# Example: accuracy might be 82% today, but 84% tomorrow, even though nothing changed.

In [4]:
# Loading the data
data = pd.read_csv('bill_authentication.csv')
data.head()

,Variance,Skewness,Curtosis,Entropy,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [5]:
data.info

<bound method DataFrame.info of       Variance  Skewness  Curtosis  Entropy  Class
0      3.62160   8.66610   -2.8073 -0.44699      0
1      4.54590   8.16740   -2.4586 -1.46210      0
2      3.86600  -2.63830    1.9242  0.10645      0
3      3.45660   9.52280   -4.0112 -3.59440      0
4      0.32924  -4.45520    4.5718 -0.98880      0
...        ...       ...       ...      ...    ...
1367   0.40614   1.34920   -1.4501 -0.55949      1
1368  -1.38870  -4.87730    6.4774  0.34179      1
1369  -3.75030 -13.45860   17.5932 -2.77710      1
1370  -3.56370  -8.38270   12.3930 -1.28230      1
1371  -2.54190  -0.65804    2.6842  1.19520      1

[1372 rows x 5 columns]>

Now We Prepare X and y (features and label), the exact datapoints we will use for the modelling

In [6]:
X = data.iloc[:, :-1].values   # the first 4 features 'Variance,	Skewness,	Curtosis,	Entropy'
y = data.iloc[:, -1].values

In [8]:
y[:5]

array([0, 0, 0, 0, 0])

In [9]:
X[:6]

array([[ 3.6216 ,  8.6661 , -2.8073 , -0.44699],
       [ 4.5459 ,  8.1674 , -2.4586 , -1.4621 ],
       [ 3.866  , -2.6383 ,  1.9242 ,  0.10645],
       [ 3.4566 ,  9.5228 , -4.0112 , -3.5944 ],
       [ 0.32924, -4.4552 ,  4.5718 , -0.9888 ],
       [ 4.3684 ,  9.6718 , -3.9606 , -3.1625 ]])

Now Train / test split the datasets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

we keep some bills hidden (test set) so we can check if the robot really learned.

Now we Scale features (very important for neural nets)

In [13]:
# we want every feature be on the same scale so the brain cells learn better.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Now we build a *simple neural network* (function so we can re-use it)

In [14]:
def create_model(input_dim,
                 units=32,
                 layers_count=2,
                 dropout_rate=0.2,
                 learning_rate=0.001):
    model = keras.Sequential()
    # input layer + first hidden layer
    model.add(layers.Dense(units, activation='relu', input_shape=(input_dim,)))
    model.add(layers.Dropout(dropout_rate))
    # additional hidden layers if asked
    for _ in range(layers_count - 1):
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))
    # output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))
    # compile
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

input_dim = X_train.shape[1]
model = create_model(input_dim)
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,249 (4.88 KB)

 Trainable params: 1,249 (4.88 KB)

 Non-trainable params: 0 (0.00 B)

units = how many little brain cells per layer, layers_count = how many layers of cells stacked, dropout = makes some cells take a nap so the model doesn’t memorize.

Train with EarlyStopping (prevents overfitting)

In [15]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

history = model.fit(
    X_train, y_train,
    validation_split=0.1,   # keep a bit of training data to validate during training
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/100
31/31 - 2s - 59ms/step - accuracy: 0.7700 - loss: 0.5525 - val_accuracy: 0.8182 - val_loss: 0.4773
Epoch 2/100
31/31 - 0s - 5ms/step - accuracy: 0.8713 - loss: 0.4128 - val_accuracy: 0.8636 - val_loss: 0.3636
Epoch 3/100
31/31 - 0s - 6ms/step - accuracy: 0.9037 - loss: 0.3165 - val_accuracy: 0.9091 - val_loss: 0.2724
Epoch 4/100
31/31 - 0s - 5ms/step - accuracy: 0.9321 - loss: 0.2305 - val_accuracy: 0.9273 - val_loss: 0.1958
Epoch 5/100
31/31 - 0s - 5ms/step - accuracy: 0.9443 - loss: 0.1770 - val_accuracy: 0.9545 - val_loss: 0.1408
Epoch 6/100
31/31 - 0s - 5ms/step - accuracy: 0.9645 - loss: 0.1263 - val_accuracy: 0.9727 - val_loss: 0.1049
Epoch 7/100
31/31 - 0s - 5ms/step - accuracy: 0.9645 - loss: 0.1051 - val_accuracy: 0.9727 - val_loss: 0.0817
Epoch 8/100
31/31 - 0s - 10ms/step - accuracy: 0.9747 - loss: 0.0796 - val_accuracy: 0.9727 - val_loss: 0.0663
Epoch 9/100
31/31 - 0s - 9ms/step - accuracy: 0.9787 - loss: 0.0664 - val_accuracy: 0.9818 - val_loss: 0.0550
Epoch 10

we let the robot practice (epochs) but stop early if it stops getting better so it doesn’t "memorize" the practice bills

Predict and evaluate whether the NN algorithm rightly learned from the datasets and can accurately predict the target values

In [16]:
# probabilities then convert to 0/1
y_prob = model.predict(X_test).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
[[153   0]
 [  0 122]]
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       153
           1     1.0000    1.0000    1.0000       122

    accuracy                         1.0000       275
   macro avg     1.0000    1.0000    1.0000       275
weighted avg     1.0000    1.0000    1.0000       275



From the confusion matrix

153 → Real bills correctly detected ✅
0 → Real bills incorrectly marked as fake ❌ (none!)
0 → Fake bills incorrectly marked as real ❌ (none!)
122 → Fake bills correctly detected ✅

This means the neural network got every single bill right in the test set — perfect prediction!

accuracy = 1.0000
Accuracy = correct predictions ÷ total predictions
all 275 bills were predicted correctly → 100% accuracy

Precision
How many predicted “real” bills are actually real
1.000
Every time the robot said “real”, it was correct!

Recall
How many actual real bills the robot caught
1.000
The robot didn’t miss any real bills.

F1-score
Balance of precision & recall
1.000
Perfect balance — the robot is super smart!

Support = number of bills in that category (153 real, 122 fake).

[[TN FP]
 [FN TP]]


How to Make the Neural Network Even Smarter


Hyperparameter tuning
we can tune the neural network “knobs” (hyperparameters):
units (neurons per layer)
layers_count (how many layers)
dropout_rate
learning_rate
batch_size and epochs

Different approaches
1. Manual tuning (simple & safe)
2. Grid search (automated, but expensive)
3. Keras Tuner (recommended for NN)

In [17]:
# Manual tuning (simple & safe)
# we try a few settings, compare validation accuracy:
for units in [16, 32]:
    for layers_count in [1, 2]:
        model = create_model(input_dim, units=units, layers_count=layers_count)
        history = model.fit(X_train, y_train, validation_split=0.1,
                            epochs=50, batch_size=32, verbose=0)
        val_acc = history.history['val_accuracy'][-1]
        print(f"units={units}, layers={layers_count} -> val_acc={val_acc:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


units=16, layers=1 -> val_acc=0.9818
units=16, layers=2 -> val_acc=1.0000
units=32, layers=1 -> val_acc=0.9818
units=32, layers=2 -> val_acc=1.0000


In [18]:
# Grid search (automated, but expensive)
# we can systematically try many combinations. Example grid
grid = {
    'units': [16, 32],          # 2 choices
    'layers_count': [1, 2],     # 2 choices
    'batch_size': [16, 32],     # 2 choices
    'epochs': [50, 100]         # 2 choices
}

we asked ourselves how many models will be trained?

Combinations = 2 × 2 × 2 × 2 = 16 different parameter combinations.

we can use 5-fold cross-validation to evaluate each combination, total training runs = 16 × 5 = 80 model training runs.

(Arithmetic shown step-by-step: 2 × 2 = 4; 4 × 2 = 8; 8 × 2 = 16 combos; 16 × 5 = 80 trainings.)

The Grid search is like trying all cookie recipes from a tiny cookbook; it can take a while.

Warning: Grid searching neural nets is slower than for small models. Prefer RandomizedSearch or Keras Tuner for bigger grids.

In [21]:
# Keras Tuner (recommended for NN) Approach
# Keras Tuner auto-searches efficiently (RandomSearch, Hyperband, Bayesian)
import kerastuner as kt

def build_model(hp):
    units = hp.Int('units', 8, 64, step=8)
    layers_count = hp.Int('layers', 1, 3)
    lr = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
    model = create_model(input_dim, units=units, layers_count=layers_count, learning_rate=lr)
    return model

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=10, executions_per_trial=1, directory='tuner_dir', project_name='bill_nn')
tuner.search(X_train, y_train, validation_split=0.1, epochs=50, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)])
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Trial 10 Complete [00h 00m 15s]
val_accuracy: 0.9727272987365723

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 02m 34s


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 24)             │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,345 (5.25 KB)

 Trainable params: 1,345 (5.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
!pip install keras-tuner --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.9 MB/s eta 0:00:00


In [22]:
model.save('bill_nn_model.h5')             # save the trained model
# To load later:
loaded = keras.models.load_model('bill_nn_model.h5')

Impact of using a Neural Network 🌟
Neural networks can capture complex patterns in the bill features.
With the right tuning they can match or beat tree-based models for some datasets.
They need more careful scaling and tuning but can be very flexible

Insights you may achieve (what to expect) 🔍
Neural nets often need feature scaling (we used StandardScaler).
Small datasets may favor simpler models (XGBoost sometimes wins on small tabular data).
Tuning learning rate, dropout, and size matters a lot.
Use confusion matrix to know whether your model is letting fake bills through (dangerous) or wrongly rejecting real bills (annoying).

What we will have achieved
Built a neural-network robot that learns to tell real vs fake bills.
Trained and tested it safely using a held-out test set.
Added protections (dropout, early stopping) so it doesn’t cheat by memorizing.
Shown how to tune the robot’s settings (by hand, grid, or using Keras Tuner).
Saved the trained model so you can use it later without retraining.